In [2]:
import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

In [3]:
!which python

/home/winegarj_test/.conda/envs/py27/bin/python


In [4]:
%%writefile mrjob_systems_test.py
#!/opt/anaconda/bin/python
from mrjob.compat import jobconf_from_env
from mrjob.job import MRJob


class MRCountLinesByFile(MRJob):

    def mapper(self, _, line):
        chars = len(line)
        words = len(line.split())
        lines = 1
        self.increment_counter('Item Counters', 'Lines', lines)
        self.increment_counter('Item Counters', 'Words', words)
        self.increment_counter('Item Counters', 'Characters', chars)
        yield "chars", chars
        yield "words", words
        yield "lines", lines

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRCountLinesByFile.run()

Overwriting mrjob_systems_test.py


In [5]:
%%writefile stuff.txt
Here I am in the altiscale
Reading the jupyter notebook

Overwriting stuff.txt


In [6]:
!python mrjob_systems_test.py stuff.txt 

No configs found; falling back on auto-configuration
Creating temp directory /tmp/mrjob_systems_test.winegarj_test.20171003.235721.867534
Running step 1 of 1...
Counters: 3
	Item Counters
		Characters=54
		Lines=2
		Words=10
Counters: 3
	Item Counters
		Characters=54
		Lines=2
		Words=10
Streaming final output from /tmp/mrjob_systems_test.winegarj_test.20171003.235721.867534/output...
"chars"	54
"lines"	2
"words"	10
Removing temp directory /tmp/mrjob_systems_test.winegarj_test.20171003.235721.867534...


In [7]:
OUTPUT_PATH = os.path.join(OUTPUT_PATH_BASE,'tests')
!hadoop fs -rm -r {OUTPUT_PATH}
!python mrjob_systems_test.py \
        -r hadoop stuff.txt \
        --output-dir={OUTPUT_PATH} \
        --no-output

17/10/03 23:57:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/winegarj_test/tests' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/winegarj_test/.Trash/Current
No configs found; falling back on auto-configuration
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/mrjob_systems_test.winegarj_test.20171003.235724.295719
Copying local files to hdfs:///user/winegarj_test/tmp/mrjob/mrjob_systems_test.winegarj_test.20171003.235724.295719/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob8955398083845759203.jar tmpDir=null
  Timeline service

In [8]:
!hadoop fs -cat {OUTPUT_PATH}/*

"chars"	54
"lines"	2
"words"	10
